# L96 Ginelli Code Redevlop
Redevoloping code to work with observers. Will then allow me to checkpoint with ease.

In [ ]:
# Standard Imports

import numpy as np
import time as tm
import pickle
from tqdm.notebook import tqdm
import xarray as xr
import sys
import os

## Working notes:

- Need to make Ginelli object. Design should be independent of integrator being used.
- Need observer of Ginelli algorithm. Think through design carefully.

## Example Observation of TL96

Below we show how to use TL96 integrator and observer in combination

In [15]:
from tangent_integrator import * # Defines integrator and observer classes for TL96

# Make Integrato and observer Objects
runner = TangentIntegratorL96(K=2, J=2)
looker = L96TrajectoryObserver(runner)

# Function for making observations
def make_observations(runner, looker, obs_num, obs_freq, noprog=False):
    """Makes observations.
    runner, integrator object.
    looker, observer object.
    obs_num, how many observations you want.
    obs_freq, adimensional time between observations"""
    for step in tqdm(np.repeat(observe_freq, obs_num), disable=noprog):
        runner.integrate(observe_freq)
        looker.look(runner)
        
make_observations(runner, looker, 20, 0.3)

# Ginelli Development

In [ ]:
def posQR(M):
    """ Returns QR decomposition of a matrix with positive diagonals on R.
    Parameter, M: Array that is being decomposed
    """
    Q, R = np.linalg.qr(M) # Performing QR decomposition
    signs = np.diag(np.sign(np.diagonal(R))) # Matrix with signs of R diagonal on the diagonal
    Q, R = np.dot(Q, signs), np.dot(signs, R) # Ensuring R Diagonal is positive
    return Q, R

class GinelliL96(TangentIntegratorL96):
    """Builds upon a tangent integrator class to perform the Ginelli algorithm"""
    
    def __init__(self, ka, kb, kc, tau=0.1, K=36, J=10, h=1, Ff=6, Fs=10, c=10, dt=0.001, save_every=10,
                 X_init=None, Y_init=None, dx_init=None, dy_init=None, noprog=True):
        
        self.save_every = save_every # how many tau steps we save
        run_save_step = (self.save_every * tau)/dt
        
        super().__init__(K=K, J=J, h=h, Ff=Ff, Fs=Fs, c=c, dt=dt, save_every=run_save_step,
                 X_init=X_init, Y_init=Y_init, dx_init=dx_init, dy_init=dy_init, noprog=noprog)
        
        # Ginelli Parameters
        self.ka, self.kb, self.kc, self.tau = ka, kb, kc, tau
    
        # Ginelli Variables
        self.P = np.random.rand(self.size, self.size) # Stretched Matrix
        eps = 0.001
        self.oldQ = eps * np.identity(self.size)
        self.oldQ[0, 1] = eps * 1
        self.R = np.random.rand(self.size, self.size)  # Stretching rates
        self._history_R = []
        self.oldA = np.identity(self.size) # Initial A
        self.oldA[0, 1] = 1

        # Lyapunov Spectra
        self.FTBLE = np.random.rand(int(self.kb), self.size)
        self.FTCLE = np.random.rand(int(self.kb), self.size)
        self._history_FTBLE = [] # For storing time series
        self._history_FTCLE = []

        # Lyapunov Vectors
        self.BLV = np.random.rand(self.size, self.size)
        self.CLV = np.random.rand(self.size, self.size)
        self._history_BLV = []
        self._history_CLV = []
        
        self.ginelli_count = 0 # Number of QR steps performed
        
    def _ginelli_step(self):
        """One QR step. Take old Q, stretch it, do a QR decomposition."""

        # Where we are in phase space before ginelli step
        phase_state = self.state[:self.size]

        # Stretching first column
        self.set_state(phase_state, self.oldQ.T[0]) # First column of Q is ic for TLE
        self.integrate(self.tau)

        # Saving Output
        self.P[:, 0] = np.append(self.dx, self.dy)

        # Stretching the rest of the columns
        for i, column in enumerate(self.oldQ.T[1:]): # Evolve each CLV
            self.set_state(phase_state, column)
            self._integrate(self.tau)
            self.P[:, i] = np.append(self.dx, self.dy) # Building P

        # QR decomposition
        self.oldQ, self.R = posQR(self.P)
        self.ginelli_count += 1

    
    @property
    def le_data(self):
        """Returns xarray of LEs and LVs"""
        if (len(self._history_X) == 0):
            print('No history to look at!')
        dic = {}
        _time = np.arange(self.ginelli_count - (len(self._history_FTBLE) - 1) * self.save_every,
                          self.ginelli_count + 1, self.save_every) * self.tau 
        # Notice we add step count. Might change run_data if you have cleared history.
        
        dic['FTBLE'] = xr.DataArray(self._history_FTBLE, dims=['time', 'le_index'], name='FTBLE',
                                coords = {'time': _time,'le_index':np.arange(1, 1 + self.size)})
        dic['FTCLE'] = xr.DataArray(self._history_FTCLE, dims=['time', 'le_index'], name='FTCLE',
                                coords = {'time': _time,'le_index':np.arange(1, 1 + self.size)})
        dic['BLV'] = xr.DataArray(self._history_BLV, dims=['time', 'component', 'le_index'], name='BLV',
                                coords = {'time': _time,'le_index':np.arange(1, 1 + self.size),
                                  'component': np.arange(self.size), 'le_index':np.arange(1, 1 + self.size)})
        dic['CLV'] = xr.DataArray(self._history_CLV, dims=['time', 'component', 'le_index'], name='CLV',
                                coords = {'time': _time,'le_index':np.arange(1, 1 + self.size),
                                  'component': np.arange(self.size), 'le_index':np.arange(1, 1 + self.size)})
        
        return xr.Dataset(dic, attrs = self.ginelli_dict)
        

In [ ]:
ginelliRunner = GinelliL96(1,1,1,1)

In [ ]:
ginelliRunner.integrate(1)

In [ ]:
ginelliRunner.le_data

In [ ]:
ginelliRunner.run_data